In [1]:
# clear variable cache
%reset -f

In [2]:
# Data
import pandas as pd
import numpy as np
import datetime

# Notebook Settings
import os

In [3]:
# set wd
os.getcwd()
os.chdir('/home/ad-frazier/data_science/MSBA_320/final_project/clean')

# set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Import GDP Data -- Dataset Core

In [4]:
df_gdp = pd.read_csv('gdp_state_annual_clean.csv')

In [5]:
df_gdp.head(2)

,state,year,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus
0,AK,1997,-62900000.00,2464700000.00,12347800000.00,25810800000.00,11061300000.00
1,AK,1998,-54700000.00,2003900000.00,12889600000.00,24227500000.00,9388700000.00


In [6]:
# Get unique states for join integrity confirmation
print('unique states:', df_gdp.state.unique().shape[0])
print('earliest year:', df_gdp.year.min())
print('earliest year:', df_gdp.year.max())

unique states: 51
earliest year: 1997
earliest year: 2020


In [7]:
df_gdp.shape

(1224, 7)

## Import Personal Income + Population

In [8]:
per_income = pd.read_csv('per_capita_personal_income-population_state_annual_clean.csv')

In [9]:
per_income.head(2)

,year,state,per_capita_personal_income,population
0,1929,AL,319.00,2644000.00
1,1929,AR,303.00,1852000.00


# Join 1: GDP + Personal Income and Population

In [10]:
df_core = df_gdp.merge(per_income, how='inner', on=['state','year'])

df_core.head(2)

,state,year,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,population
0,AL,1997,-274500000.00,6754800000.00,61083800000.00,104811900000.00,37247900000.00,21516.00,4367935.00
1,AL,1998,-292900000.00,6968300000.00,64168600000.00,110212000000.00,39368100000.00,22668.00,4404701.00


In [11]:
mov_col = df_core.pop('population')

df_core.insert(2, 'population', mov_col)

df_core.head(3)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income
0,AL,1997,4367935.00,-274500000.00,6754800000.00,61083800000.00,104811900000.00,37247900000.00,21516.00
1,AL,1998,4404701.00,-292900000.00,6968300000.00,64168600000.00,110212000000.00,39368100000.00,22668.00
2,AL,1999,4430141.00,-388400000.00,7329600000.00,67225100000.00,115680100000.00,41513700000.00,23333.00


In [12]:
df_core['per_cap_gdp_current_dollar'] = df_core['gdp_current_dollar']/df_core['population']

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar
0,AL,1997,4367935.00,-274500000.00,6754800000.00,61083800000.00,104811900000.00,37247900000.00,21516.00,23995.76
1,AL,1998,4404701.00,-292900000.00,6968300000.00,64168600000.00,110212000000.00,39368100000.00,22668.00,25021.45


In [13]:
# convert remaing gdp measures to per capita
df_core[['subsidies','taxes_on_production_and_imports_(topi)','gdp_compensation','gdp_gross_operating_surplus']] = df_core[['subsidies','taxes_on_production_and_imports_(topi)','gdp_compensation','gdp_gross_operating_surplus']].div(df_core['population'], axis=0)

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45


## Import Poverty Rate

In [14]:
poverty_rate = pd.read_csv('poverty_rate_state_annual_clean.csv')

In [15]:
poverty_rate.head(2)

,state,poverty_rate,year
0,AL,0.15,2020
1,AK,0.13,2020


## JOIN 2: df_core + poverty rate

In [16]:
def magic_merge(df1, df2):
    df1 = df1.merge(df2, how='left', on=['state','year'])
    return df1

In [17]:
df_core = magic_merge(df_core, poverty_rate)

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14


## Import gini_index

In [18]:
gini_index = pd.read_csv('gini_index_state_annual_clean.csv')

gini_index.head(2)

,state,year,gini_index
0,AL,2006,0.47
1,AL,2007,0.47


In [19]:
df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14


In [20]:
df_core = magic_merge(df_core, gini_index)

df_core.head(2)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16,NaN
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14,NaN


## Import Median Age

In [21]:
median_age = pd.read_csv('median_age_state_annual_clean.csv')

median_age.head(2)

,state,year,median_age
0,AL,2005,37.40
1,AL,2006,37.20


In [22]:
df_core = magic_merge(df_core, median_age)

df_core.sample(5)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age
1094,WA,2011,6830460.00,-206.79,4448.13,28428.00,379574900000.00,22901.58,44225.00,55570.91,0.12,0.44,37.30
1035,VA,2000,7105817.00,-96.67,2327.37,22405.24,268470900000.00,13145.91,32715.00,37781.85,0.08,NaN,NaN
973,TN,2010,6356320.00,-118.61,2959.57,21434.98,257776000000.00,16278.33,35851.00,40554.28,0.17,0.47,38.00
252,IA,2009,3032870.00,-415.71,2929.70,23281.51,136894600000.00,19341.48,37286.00,45136.98,0.11,0.43,38.00
870,PA,2003,12374658.00,-133.43,2504.13,21460.00,460270600000.00,13363.92,33170.00,37194.61,0.10,NaN,NaN


## Import Median Income

In [23]:
median_income = pd.read_csv('median_income_state_annual_clean.csv')

median_income.head(2)

,state,year,median_income
0,AL,2005,36879.00
1,AL,2006,38783.00


In [24]:
df_core = magic_merge(df_core, median_income)

### Import Mean hours worked

In [25]:
mean_hrs = pd.read_csv('hrs_worked_state_annual_clean.csv')

mean_hrs.head(5)

,state,year,mean_usual_hours
0,AL,2005,39.80
1,AL,2006,39.80
2,AL,2007,39.80
3,AL,2008,39.40
4,AL,2009,38.70


In [26]:
df_core = magic_merge(df_core, mean_hrs)

df_core.sample(15)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours
639,ND,2012,705071.00,-1232.07,5173.66,34587.86,51832600000.00,34984.42,56443.00,73514.01,0.11,0.46,36.20,53585.00,40.50
988,TX,2001,21319622.00,-166.81,2940.33,19892.85,773375500000.00,13608.93,29644.00,36275.29,0.15,NaN,NaN,NaN,NaN
1142,WV,2011,1857326.00,-84.69,2972.61,19376.03,69234100000.00,15012.23,34581.00,37276.22,0.17,0.47,41.40,38482.00,38.80
989,TX,2002,21690325.00,-114.73,3064.15,19663.20,789817600000.00,13800.74,29524.00,36413.36,0.16,NaN,NaN,NaN,NaN
620,NC,2017,10175266.00,-127.04,3427.77,27934.05,549670600000.00,22785.49,44529.00,54020.27,0.15,0.48,38.80,52752.00,39.30
479,ME,2020,1362280.00,-1787.74,4753.94,29557.29,69271800000.00,18326.41,53728.00,50849.90,0.08,NaN,NaN,NaN,NaN
829,OK,2010,3759581.00,-149.38,2670.14,20709.73,154287400000.00,17807.97,37276.00,41038.46,0.16,0.45,36.30,42072.00,39.10
341,KS,2002,2713535.00,-231.21,2466.75,19790.68,91584200000.00,11724.63,29248.00,33750.88,0.10,NaN,NaN,NaN,NaN
824,OK,2005,3548597.00,-144.51,2237.17,18064.44,125081600000.00,15091.12,31856.00,35248.18,0.16,NaN,36.50,37063.00,40.00
631,ND,2004,644705.00,-932.21,2928.47,19698.62,23259000000.00,14381.93,30107.00,36076.97,0.10,NaN,NaN,NaN,NaN


## Import Median Monthly house cost

In [27]:
monthly_house_cost = pd.read_csv('median_monthly_house_cost_state_annual_clean.csv')

monthly_house_cost.head(5)

,state,year,median_monthly_house_cost
0,AL,2005,624.00
1,AL,2006,685.00
2,AL,2007,713.00
3,AL,2008,742.00
4,AL,2009,751.00


In [28]:
df_core = magic_merge(df_core, monthly_house_cost)

df_core.sample(5)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost
587,MT,2008,976415.00,-413.25,2570.32,20049.06,37224900000.00,15917.92,35792.00,38124.06,0.13,0.45,39.10,43654.00,38.80,758.00
445,MD,2010,5791559.00,-178.90,3316.00,29921.17,314728300000.00,21284.32,49668.00,54342.59,0.11,0.44,38.00,68854.00,38.80,1672.00
344,KS,2005,2745299.00,-465.60,2989.40,21898.63,105526300000.00,14016.51,32545.00,38438.91,0.12,NaN,36.10,42920.00,39.70,776.00
238,GA,2019,10642947.00,-163.89,3594.01,31045.62,637798700000.00,25451.15,48697.00,59926.89,0.12,0.48,37.20,61980.00,39.40,1059.00
933,SC,2018,5021059.00,-99.48,3535.97,25715.27,233665300000.00,17385.30,44326.00,46537.06,0.13,0.48,39.70,52306.00,39.00,828.00


### Import House Values

In [29]:
house_value = pd.read_csv('house_value_quartiles_state_annual_clean.csv')

house_value.head(5)

,state,year,house_low_value_quartile,house_median_value,house_upper_value_quartile
0,AL,2005,59400.00,97500.00,162500.00
1,AL,2006,64200.00,107000.00,174900.00
2,AL,2007,68900.00,115600.00,187700.00
3,AL,2008,68300.00,121500.00,200600.00
4,AL,2009,67400.00,119600.00,197200.00


In [30]:
df_core = magic_merge(df_core, house_value)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile
671,NE,2020,1961455.00,-3076.19,4554.58,34235.81,133439000000.00,32316.47,56869.00,68030.62,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,ID,2013,1618042.00,-171.69,2401.42,19639.79,61835800000.00,16346.92,36095.00,38216.44,0.12,0.44,35.70,46783.00,38.30,886.00,105300.00,159000.00,240800.00
472,ME,2013,1332596.00,-145.06,3632.46,23360.42,54470100000.00,14027.43,40329.00,40875.18,0.11,0.45,44.00,46974.00,38.00,904.00,104600.00,172800.00,264100.00
572,MS,2017,2983393.00,-176.11,3124.23,20216.01,109962500000.00,13694.07,36907.00,36858.20,0.18,0.48,37.50,43529.00,39.40,595.00,67200.00,120200.00,196000.00
984,TX,1997,19740317.00,-92.21,2537.34,16055.91,610157300000.00,12408.15,23919.00,30909.19,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,SD,1999,750412.00,-998.52,2403.75,14697.53,21078700000.00,11986.75,25306.00,28089.50,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1092,WA,2009,6667426.00,-201.44,4105.99,27504.17,351245000000.00,21272.03,41591.00,52680.75,0.12,0.44,36.90,56548.00,38.10,1420.00,192900.00,287200.00,426300.00
1143,WV,2012,1858745.00,-86.29,3021.29,20083.77,70322200000.00,14814.46,35453.00,37833.16,0.17,0.46,41.50,40196.00,38.90,494.00,54500.00,100400.00,168500.00
257,IA,2014,3121583.00,-328.97,3391.04,27013.95,173095900000.00,25375.30,44699.00,55451.32,0.10,0.44,38.20,53712.00,38.60,843.00,81900.00,133100.00,205600.00
13,AL,2010,4787998.00,-96.93,2311.34,20218.05,177249200000.00,14587.02,33946.00,37019.48,0.17,0.47,37.80,40474.00,38.60,772.00,NaN,NaN,NaN


### Rent Quartiles

In [31]:
rent_quarts = pd.read_csv('rent_quartiles_state_annual_clean.csv')

rent_quarts.head(5)

,state,year,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile
0,AL,2005,272.00,394.00,541.00
1,AL,2006,282.00,410.00,562.00
2,AL,2007,291.00,427.00,591.00
3,AL,2008,314.00,447.00,619.00
4,AL,2009,322.00,468.00,637.00


In [32]:
df_core = magic_merge(df_core, rent_quarts)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile
797,OH,2002,11407889.00,-102.81,2327.02,20719.59,411034500000.00,13086.93,29693.00,36030.72,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
541,MO,2010,5996166.00,-171.26,2586.50,23754.06,259816100000.00,17161.08,37118.00,43330.37,0.15,0.46,38.00,44301.00,38.40,876.00,NaN,NaN,NaN,NaN,NaN,NaN
1143,WV,2012,1858745.00,-86.29,3021.29,20083.77,70322200000.00,14814.46,35453.00,37833.16,0.17,0.46,41.50,40196.00,38.90,494.00,54500.00,100400.00,168500.00,326.00,448.00,600.00
529,MO,1998,5521765.00,-129.96,1984.44,17501.38,169483400000.00,11337.86,25540.00,30693.70,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
662,NE,2011,1843774.00,-449.73,3411.75,25945.10,99921100000.00,25286.67,45674.00,54193.79,0.10,0.45,36.30,50296.00,39.10,909.00,NaN,NaN,NaN,NaN,NaN,NaN
396,LA,2009,4491648.00,-363.59,3099.20,22642.02,204906900000.00,20241.90,36930.00,45619.54,0.14,0.47,35.60,42492.00,40.00,694.00,73700.00,135400.00,206100.00,379.00,566.00,763.00
389,LA,2002,4497267.00,-122.99,2477.04,16608.00,139800900000.00,12123.70,26289.00,31085.75,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,DC,2007,574404.00,-1089.65,6391.67,112629.79,93785400000.00,45342.48,59824.00,163274.28,0.18,0.54,34.90,54317.00,40.70,1665.00,310000.00,450900.00,721200.00,590.00,871.00,1278.00
900,RI,2009,1053646.00,-183.27,3709.97,24848.67,47702200000.00,16898.09,40610.00,45273.46,0.13,0.46,39.40,54119.00,37.70,1542.00,204200.00,267100.00,368700.00,527.00,748.00,954.00
816,OK,1997,3372917.00,-92.65,1647.30,12936.87,79044400000.00,8943.48,21072.00,23435.03,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Import Occupant Ratio

In [33]:
house_occ_ratio = pd.read_csv('household_occupants_ratio_state_annual_clean.csv')

house_occ_ratio.head(5)

,state,year,household_renter_occ_ratio,household_owner_occ_ratio
0,AL,2005,0.28,0.72
1,AL,2006,0.26,0.74
2,AL,2007,0.27,0.73
3,AL,2008,0.27,0.73
4,AL,2009,0.29,0.71


In [34]:
df_core = magic_merge(df_core, house_occ_ratio)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio
425,MA,2014,6819480.00,-245.47,3749.54,39675.66,470468500000.00,25809.18,58924.00,68988.91,0.14,0.49,39.40,69160.00,37.80,1599.00,228300.00,338900.00,487600.00,625.00,972.00,1389.00,0.34,0.66
195,FL,2000,16047515.00,-78.47,2852.74,17311.68,490429800000.00,10475.15,29466.00,30561.11,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
916,SC,2001,4064995.00,-86.10,2092.84,16978.50,119478100000.00,10406.70,25737.00,29391.94,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,KY,2016,4457599.00,-122.98,3197.08,24540.77,196484900000.00,16463.77,39750.00,44078.64,0.15,0.48,38.70,46659.00,38.90,756.00,81400.00,135600.00,212500.00,400.00,555.00,740.00,0.32,0.68
648,NE,1997,1686418.00,-302.71,2181.19,16187.68,49819200000.00,11475.27,24915.00,29541.43,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,MI,2010,9880076.00,-105.56,3047.64,22106.15,390043000000.00,14429.50,35760.00,39477.73,0.16,0.45,39.00,45413.00,37.20,969.00,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.75
641,ND,2014,744395.00,-1264.25,6209.34,39176.51,59926300000.00,36381.76,57523.00,80503.36,0.10,0.47,35.10,59029.00,40.90,743.00,86400.00,161800.00,251200.00,473.00,644.00,868.00,0.31,0.69
260,IA,2017,3162621.00,-372.73,3773.42,29318.72,183549800000.00,25317.83,47286.00,58037.24,0.07,0.44,38.30,58570.00,39.00,879.00,90300.00,149100.00,232400.00,465.00,628.00,808.00,0.25,0.75
549,MO,2018,6128594.00,-215.47,3135.42,30153.60,319393900000.00,19041.79,47230.00,52115.36,0.12,0.47,38.80,54478.00,38.90,891.00,97600.00,162600.00,256600.00,476.00,639.00,837.00,0.30,0.70
274,ID,2007,1505105.00,-139.92,2306.55,19924.86,53934500000.00,13742.90,32707.00,35834.38,0.10,0.44,34.30,46253.00,39.20,909.00,119800.00,178100.00,278000.00,415.00,564.00,729.00,0.26,0.74


### Household Occupied Binary

In [35]:
house_occ_bool = pd.read_csv('household_occupancy_binary_state_annual_clean.csv')

house_occ_bool.head(5)

,state,year,households_occupied_ratio
0,AK,2005,0.85
1,AK,2006,0.83
2,AK,2007,0.84
3,AK,2008,0.84
4,AK,2009,0.83


In [36]:
df_core = magic_merge(df_core, house_occ_bool)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio
384,LA,1997,4421071.00,-87.26,2029.78,13658.43,116313200000.00,10707.88,21352.00,26308.83,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
652,NE,2001,1719836.00,-789.90,2372.61,19938.18,59607200000.00,13137.76,30275.00,34658.65,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,SD,2001,757972.00,-996.08,2450.09,16078.30,23412100000.00,13355.51,27645.00,30887.82,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,AR,2006,2821761.00,-221.21,2577.26,18112.87,95875100000.00,13508.16,29617.00,33977.04,0.18,0.46,37.10,36599.00,39.80,611.00,58100.00,93900.00,154300.00,296.00,407.00,537.00,0.30,0.70,0.87
552,MS,1997,2777004.00,-96.94,1692.98,11748.02,58352700000.00,7668.80,19221.00,21012.83,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
921,SC,2006,4357847.00,-112.21,2733.21,19960.04,153647700000.00,12676.65,30984.00,35257.71,0.11,0.46,37.10,41100.00,39.50,772.00,73900.00,122400.00,197900.00,349.00,491.00,653.00,0.28,0.72,0.84
512,MN,2005,5119598.00,-374.93,3107.59,27034.92,245380500000.00,18162.09,38065.00,47929.64,0.08,NaN,36.70,52024.00,38.60,1066.00,131400.00,198800.00,285500.00,441.00,635.00,817.00,0.20,0.80,0.90
304,IL,2013,12960987.00,-236.73,4190.98,30886.68,741121500000.00,22340.01,47047.00,57180.95,0.14,0.48,37.20,56210.00,38.20,1156.00,98000.00,169600.00,277100.00,546.00,772.00,1045.00,0.31,0.69,0.90
302,IL,2011,12892740.00,-240.99,3850.76,29280.13,692880800000.00,20852.04,44194.00,53741.94,0.14,0.47,36.80,53234.00,38.20,1237.00,NaN,NaN,NaN,NaN,NaN,NaN,0.30,0.70,0.89
590,MT,2011,998188.00,-443.80,2681.06,20665.45,40995100000.00,18166.82,38558.00,41069.52,0.17,0.44,39.70,44222.00,37.70,809.00,NaN,NaN,NaN,NaN,NaN,NaN,0.30,0.70,0.83


### Import Student Funding(revenue)


In [37]:
stud_rev = pd.read_csv('tot_revenue_student_state_annual_clean.csv')

stud_rev.head(5)

,state,year,per_pupil_revenue
0,AL,2018,11415
1,AK,2018,19553
2,AZ,2018,10396
3,AR,2018,11733
4,CA,2018,16014


In [38]:
df_core = magic_merge(df_core, stud_rev)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue
196,FL,2001,16356966.00,-107.99,2837.44,18029.60,519058600000.00,10974.13,30418.00,31733.18,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7178.00
534,MO,2003,5709403.00,-172.45,2244.61,20702.08,205588500000.00,13234.52,30085.00,36008.76,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8762.00
228,GA,2009,9620846.00,-139.10,2667.19,23010.75,409590800000.00,17034.42,34274.00,42573.26,0.18,0.47,34.60,47590.00,39.00,1092.00,103200.00,162800.00,257400.00,444.00,637.00,838.00,0.32,0.68,0.85,10695.00
75,CA,2000,33987977.00,-140.10,2460.46,22602.08,1356975400000.00,15002.72,33410.00,39925.16,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8306.00
433,MD,1998,5204464.00,-109.75,2072.43,19050.18,169982100000.00,11647.96,31483.00,32660.83,0.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8086.00
615,NC,2012,9722105.00,-157.47,3157.55,23889.03,445095300000.00,18892.67,39103.00,45781.78,0.17,0.47,37.80,45150.00,38.80,903.00,91800.00,150100.00,239200.00,439.00,606.00,813.00,0.34,0.66,0.85,8632.00
131,CT,2008,3545579.00,-143.84,4187.84,35017.47,238936800000.00,28328.63,60970.00,67390.07,0.08,0.49,39.30,68595.00,38.60,1722.00,217900.00,306000.00,456900.00,612.00,813.00,1067.00,0.27,0.73,0.92,17404.00
373,KY,2010,4349138.00,-143.55,2799.22,21048.93,166408700000.00,14557.83,33409.00,38262.46,0.18,0.47,38.00,40062.00,38.60,741.00,NaN,NaN,NaN,NaN,NaN,NaN,0.30,0.70,0.87,10389.00
996,TX,2009,24801761.00,-142.16,3705.08,23810.70,1169607700000.00,19784.62,37269.00,47158.25,0.17,0.47,33.10,48259.00,39.70,1005.00,78100.00,125800.00,199000.00,478.00,632.00,826.00,0.34,0.66,0.88,10318.00
1150,WV,2019,1800430.00,-74.20,3481.50,23417.07,79139700000.00,17131.69,42378.00,43956.00,0.14,0.46,42.90,48850.00,39.30,533.00,68700.00,124600.00,204400.00,394.00,551.00,717.00,0.25,0.75,0.81,NaN


### Import per student spending

In [39]:
stud_spend = pd.read_csv('tot_spend_student_state_annual_clean.csv')

stud_spend.head(5)

,state,year,per_pupil_expenditure
0,AL,2018,10107
1,AK,2018,18393
2,AZ,2018,8773
3,AR,2018,10412
4,CA,2018,13831


In [40]:
df_core = magic_merge(df_core, stud_spend)

df_core.sample(15)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure
632,ND,2005,646089.00,-1453.98,3146.78,20683.53,24700000000.00,15853.70,31967.00,38230.03,0.11,NaN,39.10,41030.00,40.70,599.00,51800.00,88600.00,135800.00,306.00,422.00,551.00,0.27,0.73,0.89,9748.00,8728.00
937,SD,1998,746058.00,-587.09,2279.05,13781.37,20228100000.00,11640.11,24277.00,27113.31,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6257.00,5259.00
384,LA,1997,4421071.00,-87.26,2029.78,13658.43,116313200000.00,10707.88,21352.00,26308.83,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5784.00,5187.00
795,OH,2000,11363543.00,-117.76,2218.75,20086.65,392277200000.00,12333.04,28598.00,34520.68,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9073.00,7571.00
1122,WI,2015,5794088.00,-157.71,3585.28,29570.19,307507900000.00,20074.95,46548.00,53072.70,0.11,0.44,39.40,55638.00,38.50,1034.00,112900.00,168300.00,250900.00,522.00,669.00,850.00,0.30,0.70,0.87,13033.00,11664.00
226,GA,2007,9349988.00,-122.43,2880.52,24268.80,419606300000.00,17850.85,35680.00,44877.74,0.14,0.46,34.80,49136.00,40.10,1076.00,106900.00,164500.00,252200.00,427.00,614.00,798.00,0.30,0.70,0.86,11319.00,9718.00
1080,WA,1997,5674747.00,-135.69,3141.88,18383.22,193825900000.00,12766.45,27405.00,34155.87,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6957.00,6040.00
680,NH,2005,1298492.00,-129.23,3241.84,24596.61,56952600000.00,16151.35,40619.00,43860.57,0.06,NaN,39.50,56768.00,39.10,1253.00,166900.00,240100.00,348800.00,559.00,757.00,962.00,0.23,0.77,0.85,11489.00,10396.00
1012,UT,2001,2283715.00,-106.62,2130.08,18371.64,73057600000.00,11595.62,24925.00,31990.68,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5983.00,4900.00
1142,WV,2011,1857326.00,-84.69,2972.61,19376.03,69234100000.00,15012.23,34581.00,37276.22,0.17,0.47,41.40,38482.00,38.80,508.00,NaN,NaN,NaN,NaN,NaN,NaN,0.26,0.74,0.83,12573.00,11579.00


### Import Education Level

In [41]:
edu_level = pd.read_csv('edu_level_state_annual_clean.csv')

edu_level.head(5)

,state,year,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
0,AL,2008,210308.00,439587.00,801711.00,170625.00,3078499.00
1,AL,2009,218551.00,447402.00,797598.00,168782.00,3115982.00
2,AL,2010,225382.00,439863.00,817024.00,182474.00,3161521.00
3,AL,2011,236162.00,443488.00,818373.00,190977.00,3193078.00
4,AL,2012,238555.00,472037.00,826740.00,193901.00,3209646.00


In [42]:
df_core = magic_merge(df_core, edu_level)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
472,ME,2013,1332596.00,-145.06,3632.46,23360.42,54470100000.00,14027.43,40329.00,40875.18,0.11,0.45,44.00,46974.00,38.00,904.00,104600.00,172800.00,264100.00,493.00,668.00,875.00,0.27,0.73,0.76,14517.00,13267.00,91720.00,172498.00,268512.00,67663.00,953962.00
531,MO,2000,5607285.00,-197.69,2107.56,19422.63,186650000000.00,11954.56,27941.00,33287.05,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7781.00,6657.00,NaN,NaN,NaN,NaN,NaN
497,MI,2014,9974630.00,-106.85,3080.57,25215.97,451773200000.00,17102.53,41207.00,45292.23,0.15,0.46,39.60,49847.00,37.80,887.00,71600.00,125700.00,205800.00,497.00,641.00,823.00,0.28,0.72,0.84,12649.00,10956.00,618746.00,1105362.00,1709421.00,529318.00,6680525.00
130,CT,2007,3527270.00,-139.46,4219.64,34766.52,234540700000.00,27646.82,57684.00,66493.55,0.09,0.48,39.10,65967.00,38.80,1616.00,211300.00,309200.00,460700.00,588.00,783.00,1007.00,0.26,0.74,0.92,16585.00,14610.00,NaN,NaN,NaN,NaN,NaN
1020,UT,2009,2723421.00,-116.47,2404.99,22386.11,113985400000.00,17179.17,31858.00,41853.76,0.10,0.41,28.90,55117.00,36.60,1200.00,163800.00,224700.00,315500.00,542.00,700.00,933.00,0.26,0.74,0.91,7925.00,6452.00,143646.00,304181.00,349117.00,101453.00,1573179.00
608,NC,2005,8705407.00,-212.68,2672.52,21691.13,355469700000.00,16682.24,32345.00,40833.21,0.13,NaN,36.20,40729.00,39.70,831.00,82500.00,127600.00,195600.00,363.00,496.00,648.00,0.30,0.70,0.87,7863.00,7396.00,NaN,NaN,NaN,NaN,NaN
412,MA,2001,6397634.00,-159.50,2334.25,28335.43,296868700000.00,15892.69,39872.00,46402.89,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11319.00,10232.00,NaN,NaN,NaN,NaN,NaN
659,NE,2008,1796378.00,-466.66,3279.43,25930.23,85628000000.00,18924.02,40535.00,47667.03,0.11,0.43,36.30,49693.00,39.80,917.00,81900.00,126500.00,180600.00,364.00,506.00,662.00,0.27,0.73,0.90,12274.00,10846.00,103164.00,213091.00,299334.00,66587.00,1151267.00
893,RI,2002,1065995.00,-110.41,2828.34,20277.02,37809200000.00,12473.60,32807.00,35468.46,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10960.00,10349.00,NaN,NaN,NaN,NaN,NaN
1065,VT,2006,622892.00,-144.97,3801.62,22622.86,24331500000.00,12782.79,37008.00,39062.15,0.08,0.42,40.60,47665.00,38.60,1055.00,127300.00,193000.00,279700.00,460.00,624.00,800.00,0.23,0.77,0.82,15107.00,13629.00,NaN,NaN,NaN,NaN,NaN


In [43]:
# convert edu level to percent of population
df_core[['edu_associate_degree','edu_bachelor_degree','edu_high_school','edu_master_degree']] = df_core[['edu_associate_degree','edu_bachelor_degree','edu_high_school','edu_master_degree']].div(df_core['edu_total'], axis=0)

df_core.sample(5)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total
237,GA,2018,10532566.00,-129.22,3440.06,29968.17,602340100000.00,23909.33,46824.00,57188.35,0.15,0.48,36.80,58756.00,39.40,1024.00,114600.00,189900.00,308100.00,550.00,823.00,1127.00,0.35,0.65,0.88,12766.00,11203.00,0.08,0.20,0.23,0.09,6978172.00
774,NY,2003,19175939.00,-251.99,3469.53,25862.72,912474600000.00,18504.10,37576.00,47584.35,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14176.00,12638.00,NaN,NaN,NaN,NaN,NaN
471,ME,2012,1330899.00,-145.92,3561.28,22820.82,53679700000.00,14097.31,40411.00,40333.41,0.13,0.45,43.50,46709.00,37.80,933.00,104500.00,172300.00,263500.00,502.00,668.00,863.00,0.26,0.74,0.77,13917.00,12655.00,0.09,0.18,0.29,0.07,947017.00
884,PA,2017,12945337.00,-152.54,3864.50,31513.67,745011300000.00,22324.93,52499.00,57550.55,0.11,0.48,40.80,59195.00,38.60,996.00,110000.00,181200.00,290000.00,531.00,743.00,1010.00,0.28,0.72,0.88,18724.00,16377.00,0.08,0.19,0.31,0.09,8960842.00
544,MO,2013,6044003.00,-199.83,2759.50,25580.65,280571000000.00,18281.08,40569.00,46421.39,0.17,0.46,38.10,46931.00,38.50,837.00,80300.00,133200.00,207500.00,420.00,566.00,745.00,0.30,0.70,0.87,11380.00,9938.00,0.08,0.17,0.27,0.07,4047283.00


### Import PCE Values

In [44]:
pce = pd.read_csv('pce_function_state_annual_clean.csv')

pce.head(5)

,year,state,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
0,1997,AK,552000000.00,193700000.00,255200000.00,848300000.00,1450000000.00,1091000000.00,701300000.00,1965500000.00,2457500000.00,1146800000.00,14205600000.00,1451200000.00,1711600000.00
1,1997,AL,3457300000.00,1875300000.00,1156500000.00,5026600000.00,7747300000.00,4278700000.00,3993800000.00,14574600000.00,13361000000.00,4375700000.00,77070900000.00,6744600000.00,9454500000.00
2,1997,AR,2186700000.00,881100000.00,715600000.00,2953100000.00,3952400000.00,2370000000.00,2245300000.00,7505800000.00,7595600000.00,2464000000.00,41941200000.00,3290300000.00,5312800000.00
3,1997,AZ,3830400000.00,1930400000.00,1600000000.00,6674700000.00,8135900000.00,6703500000.00,5342900000.00,13544900000.00,20212800000.00,6564900000.00,96365600000.00,8586500000.00,12454400000.00
4,1997,CA,33257500000.00,15812800000.00,12700500000.00,51494500000.00,54740400000.00,42657600000.00,32307800000.00,106739900000.00,132581500000.00,42047000000.00,676652200000.00,69924000000.00,75773900000.00


In [45]:
# divide PCE values to get per capita PCE
pce.iloc[:, 2:] = pce.iloc[:, 2:].div(df_core['population'], axis=0)

pce.sample(10)

,year,state,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
534,2007,MO,967.68,771.81,624.67,2321.87,2322.73,1945.16,1390.32,6278.73,5282.16,2004.50,31578.94,2951.80,3829.35
812,2012,WA,628.86,638.77,393.56,1489.32,1827.72,1304.01,834.15,4326.88,4106.01,1454.94,21845.26,2085.25,2180.68
264,2002,FL,11064.04,7681.93,4959.46,30079.69,29014.99,24008.97,20239.96,63183.42,77628.45,24028.99,364688.41,32069.73,38200.84
456,2005,WI,4009.17,1661.65,2740.81,9692.42,9867.83,7126.46,6019.09,26580.40,23399.83,7758.93,128033.49,10858.29,13849.35
229,2001,MS,236.29,99.38,100.16,288.41,519.14,312.61,264.70,1112.94,1011.69,324.38,5436.95,378.75,693.71
1086,2018,IN,1148.70,543.16,802.41,2831.84,3245.96,2647.38,1798.23,10265.68,7212.79,2290.18,41577.28,3122.42,4297.18
662,2009,WY,371.03,186.41,187.44,992.75,897.24,677.47,387.79,1837.32,1867.58,564.98,10061.32,770.43,1179.48
1119,2018,WI,1057.92,553.88,809.06,3226.00,2966.08,2492.84,1639.25,9608.67,7151.36,2348.65,41507.71,3631.85,4146.20
386,2004,NE,356.73,156.65,231.82,973.30,821.44,600.59,452.61,2048.82,1681.04,617.56,10554.29,827.07,1434.54
1170,2019,WI,10196.10,5532.47,8338.97,32310.05,29627.08,25289.17,16461.21,98578.59,71547.60,23821.16,419013.44,36859.73,41118.32


In [46]:
df_core = magic_merge(df_core, pce)

df_core.head(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5535.00,4849.00,NaN,NaN,NaN,NaN,NaN,784.91,425.75,262.56,1141.19,1758.87,971.39,906.71,3308.87,3033.35,993.42,17497.42,1531.23,2146.46
1,AL,1998,4404701.00,-66.50,1582.01,14568.21,110212000000.00,8937.75,22668.00,25021.45,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5975.00,5188.00,NaN,NaN,NaN,NaN,NaN,689.58,369.26,235.94,1023.88,1491.69,846.61,787.43,2883.60,2656.01,872.42,15261.47,1330.15,1841.18
2,AL,1999,4430141.00,-87.67,1654.48,15174.48,115680100000.00,9370.74,23333.00,26112.06,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6523.00,5638.00,NaN,NaN,NaN,NaN,NaN,831.52,446.36,290.84,1271.21,1785.00,1030.55,946.29,3448.67,3214.53,1089.07,18586.66,1614.90,2310.11
3,AL,2000,4452173.00,-81.92,1767.36,15669.74,119851700000.00,9564.63,24306.00,26919.82,0.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6503.00,5885.00,NaN,NaN,NaN,NaN,NaN,6807.40,3766.86,2513.74,11201.87,14512.09,8486.88,7761.44,29142.73,27368.72,9099.52,156560.54,13348.79,19776.32
4,AL,2001,4467634.00,-106.50,1791.62,16124.51,122915500000.00,9702.81,25057.00,27512.44,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6956.00,6029.00,NaN,NaN,NaN,NaN,NaN,207.49,116.88,79.26,337.49,466.71,263.61,237.65,961.09,894.23,289.32,4958.17,412.16,603.12
5,AL,2002,4480089.00,-112.65,1796.88,16551.52,127505000000.00,10224.60,25648.00,28460.37,0.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6971.00,6300.00,NaN,NaN,NaN,NaN,NaN,1267.11,723.60,497.51,2112.79,2915.46,1653.28,1503.17,6401.77,5608.76,1863.97,31402.34,2607.20,3696.54
6,AL,2003,4503491.00,-103.85,1872.92,17116.72,134152600000.00,10902.79,26673.00,29788.58,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7349.00,6581.00,NaN,NaN,NaN,NaN,NaN,307.70,173.96,125.82,543.70,721.28,424.39,376.49,1628.41,1393.34,474.17,7902.65,672.02,934.59
7,AL,2004,4530729.00,-91.91,1985.49,18007.90,147715200000.00,12701.49,28434.00,32602.97,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8028.00,7073.00,NaN,NaN,NaN,NaN,NaN,1424.22,786.67,579.12,2526.43,3301.75,1990.59,1770.43,7690.27,6486.72,2195.83,37033.53,3232.39,4428.33
8,AL,2005,4569805.00,-122.65,2169.35,18897.26,158846800000.00,13816.13,29949.00,34760.08,0.17,NaN,37.40,36879.00,39.80,624.00,59400.00,97500.00,162500.00,272.00,394.00,541.00,0.28,0.72,0.86,8555.00,7683.00,NaN,NaN,NaN,NaN,NaN,701.09,366.89,286.39,1270.43,1599.91,1001.68,889.83,3861.88,3256.06,1076.11,18474.42,1614.23,2243.87
9,AL,2006,4628981.00,-108.77,2283.72,19690.73,166469000000.00,14096.69,31474.00,35962.34,0.14,0.47,37.20,38783.00,39.80,685.00,64200.00,107000.00,174900.00,282.00,410.00,562.00,0.26,0.74,0.85,9548.00,8398.00,NaN,NaN,NaN,NaN,NaN,3545.17,1837.30,1468.31,6467.49,7979.36,5216.88,4534.52,19887.03,16940.22,5463.91,94734.12,8341.73,11509.44


### Import Business Applications data

In [47]:
busi_apps = pd.read_csv('business_apps_state_annual_clean.csv')

busi_apps.head(5)

,sa,geo,year,naics_sector,business_applications,business_applications_from_corporations,high‐propensity_business_applications_ba_wba___business_applications_with_planned_wages
0,A,AK,2005,Total for All NAICS,4770.00,1028.00,2708.00
1,A,AK,2006,Total for All NAICS,5218.00,1142.00,2961.00
2,A,AK,2007,Total for All NAICS,5197.00,1024.00,2855.00
3,A,AK,2008,Total for All NAICS,4902.00,749.00,2428.00
4,A,AK,2009,Total for All NAICS,4452.00,607.00,2122.00


In [48]:
busi_apps = busi_apps.rename(columns={'high‐propensity_business_applications_ba_wba___business_applications_with_planned_wages':'high_propensity_business_application',
                                      'geo':'state'})

In [49]:
df_core = magic_merge(df_core, busi_apps)

df_core.sample(10)

,state,year,population,subsidies,taxes_on_production_and_imports_(topi),gdp_compensation,gdp_current_dollar,gdp_gross_operating_surplus,per_capita_personal_income,per_cap_gdp_current_dollar,poverty_rate,gini_index,median_age,median_income,mean_usual_hours,median_monthly_house_cost,house_low_value_quartile,house_median_value,house_upper_value_quartile,rent_low_value_quartile,rent_median_value,rent_upper_value_quartile,household_renter_occ_ratio,household_owner_occ_ratio,households_occupied_ratio,per_pupil_revenue,per_pupil_expenditure,edu_associate_degree,edu_bachelor_degree,edu_high_school,edu_master_degree,edu_total,pce_clothing_footwear_and_related_services,pce_communication,pce_education,pce_financial_services_and_insurance,pce_food_and_beverages_purchased_for_off-premises_consumption,pce_food_services_and_accommodations,pce_furnishings_household_equipment_and_routine_household_maintenance,pce_health,pce_housing_utilities_and_fuels,pce_other_goods_and_services,pce_total,pce_recreation,pce_transportation,sa,naics_sector,business_applications,business_applications_from_corporations,high_propensity_business_application
1094,WA,2011,6830460.00,-206.79,4448.13,28428.00,379574900000.00,22901.58,44225.00,55570.91,0.12,0.44,37.30,56835.00,37.80,1378.00,NaN,NaN,NaN,NaN,NaN,NaN,0.35,0.65,0.91,11330.00,9617.00,0.10,0.20,0.20,0.08,4583426.00,2557.75,2514.12,1598.21,6080.78,7442.49,5104.22,3381.66,17264.93,16902.23,5940.66,87928.52,8192.55,8845.24,A,Total for All NAICS,48653.00,8308.00,22122.00
200,FL,2005,17842038.00,-150.54,3842.46,21352.05,705851900000.00,14517.20,35999.00,39561.17,0.11,NaN,39.50,42433.00,40.20,925.00,112500.00,189500.00,320100.00,524.00,692.00,884.00,0.30,0.70,0.85,9277.00,7812.00,NaN,NaN,NaN,NaN,NaN,2465.38,1668.53,1197.57,7539.75,6816.53,6079.09,4987.33,15008.35,17879.24,5834.14,86933.18,8005.48,9162.78,A,Total for All NAICS,283938.00,158067.00,200411.00
0,AL,1997,4367935.00,-62.84,1546.45,13984.59,104811900000.00,8527.58,21516.00,23995.76,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5535.00,4849.00,NaN,NaN,NaN,NaN,NaN,784.91,425.75,262.56,1141.19,1758.87,971.39,906.71,3308.87,3033.35,993.42,17497.42,1531.23,2146.46,NaN,NaN,NaN,NaN,NaN
82,CA,2007,36250311.00,-164.92,3433.52,28404.11,1898902000000.00,20710.35,43385.00,52383.05,0.13,0.47,34.70,59948.00,39.10,1850.00,352200.00,532300.00,744000.00,725.00,984.00,1366.00,0.40,0.60,0.92,11228.00,9706.00,NaN,NaN,NaN,NaN,NaN,9056.74,5883.45,4877.59,19399.19,17484.88,15582.65,11060.51,39173.89,44704.96,15748.82,239609.21,25324.67,25377.96,A,Total for All NAICS,288549.00,108308.00,174220.00
319,IN,2004,6233007.00,-158.38,2420.50,20965.00,239743600000.00,15236.43,30791.00,38463.55,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11043.00,8919.00,NaN,NaN,NaN,NaN,NaN,1356.52,543.15,708.98,2284.93,3056.82,2308.90,1872.12,7453.59,6566.86,2295.47,37071.20,3263.16,4365.55,NaN,NaN,NaN,NaN,NaN
1170,WY,2015,583645.00,-169.97,6342.73,31368.55,38426900000.00,28298.19,57846.00,65839.51,0.10,0.44,36.50,60214.00,40.70,902.00,142100.00,212500.00,294700.00,536.00,697.00,948.00,0.31,0.69,0.85,21569.00,16431.00,0.11,0.17,0.24,0.07,388747.00,130.39,77.75,67.20,483.52,310.28,253.23,137.35,749.02,678.19,193.24,3842.84,277.73,438.04,A,Total for All NAICS,9839.00,1773.00,3693.00
102,CO,2003,4528732.00,-191.09,2629.46,24024.43,196126500000.00,16844.34,34856.00,43307.16,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8639.00,7478.00,NaN,NaN,NaN,NaN,NaN,391.65,223.82,220.13,754.54,786.60,719.99,534.02,1609.83,1921.62,681.12,10314.70,1174.89,1158.36,NaN,NaN,NaN,NaN,NaN
955,SD,2016,860650.00,-1165.75,4192.18,26382.62,49151400000.00,27700.58,49192.00,57109.63,0.14,0.45,36.80,54467.00,39.70,834.00,92600.00,160700.00,242700.00,414.00,601.00,781.00,0.29,0.71,0.87,11592.00,10117.00,0.12,0.20,0.25,0.06,565894.00,407.58,293.40,244.39,1067.27,1086.73,994.97,611.80,3913.81,2339.62,807.49,14888.25,1217.71,1269.72,A,Total for All NAICS,6335.00,645.00,2557.00
647,ND,2020,778

In [50]:
df_core.shape

(1176, 50)

In [51]:
df_core.to_csv('../core_df.csv', index=False)

### correlation explore

In [52]:
corr = df_core.corr(method='pearson').abs()
s = corr.unstack()
df_corr = pd.DataFrame(s.sort_values(kind="quicksort")).reset_index()

df_corr.to_csv('first_pass_corr.csv', index=False)